Code to convert prediction Tiff to Binary Image using Thresholding  

In [ ]:
Below Finalized code for Threshoding 

In [23]:
from osgeo import gdal
import numpy as np

# Define the input and output file paths
input_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif'
output_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image.tif'

# Load the input TIFF image
dataset = gdal.Open(input_path)
if dataset is None:
    print("Failed to open the input TIFF image.")
    exit()

# Read the image data
image_data = dataset.ReadAsArray()

# Select a specific band (assuming grayscale image)
band_index = 0  # Adjust this value if the grayscale band is different
image_band = image_data[band_index]

# Apply thresholding
threshold_value = 100  # Adjust this value as per your requirement
binary_image_data = np.where(image_band >= threshold_value, 255, 0)

# Create the output dataset
driver = gdal.GetDriverByName('GTiff')
output_dataset = driver.Create(output_path, dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Byte)
output_dataset.GetRasterBand(1).WriteArray(binary_image_data)

# Set the geotransform and projection
output_dataset.SetGeoTransform(dataset.GetGeoTransform())
output_dataset.SetProjection(dataset.GetProjection())

# Close the datasets
output_dataset = None
dataset = None

print("Binary image created successfully.")

Binary image created successfully.


Script to execute ACM for polygonization InputBinary file output is a polygon shape file. 

In [5]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import label, regionprops
from skimage.segmentation import find_boundaries, active_contour
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Open the binary TIFF file
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image_split1.tif') as src:
    binary_image = src.read(1)
    transform = src.transform
    crs = src.crs

# Perform Connected Component Analysis
labeled_image, num_labels = label(binary_image, connectivity=2, background=0, return_num=True)
regions = regionprops(labeled_image, intensity_image=binary_image)

# Create an empty list to store individual polygons
polygons = []

# Thresholding parameters
threshold_low = 0.1
threshold_high = 0.5

# Active contours parameters
snake_iterations = 300
snake_alpha = 0.01
snake_beta = 0.1
snake_gamma = 0.001

# Iterate over the regions and extract properties
for region in regions:
    label_value = region.label
    bbox = region.bbox  # Bounding box of the region (min_row, min_col, max_row, max_col)

    # Extract boundaries of the region
    boundaries = find_boundaries(labeled_image == label_value, mode='outer')

    # Calculate the threshold level
    threshold = threshold_low + (threshold_high - threshold_low) * (np.mean(region.intensity_image) / 255)

    # Apply active contours model to extract contour
    snake_init = np.array(region.coords)
    snake = active_contour(boundaries, snake_init, alpha=snake_alpha, beta=snake_beta, gamma=snake_gamma,
                           coordinates='rc', max_num_iter=100)

    # Convert contour coordinates to Polygon format
    contour_coords = np.flip(snake, axis=1)  # Flip the coordinates
    contour_coords[:, 0] += bbox[1]  # Adjust x-coordinates
    contour_coords[:, 1] += bbox[0]  # Adjust y-coordinates
    
    # Check if there are at least 3 points in the contour
    if len(contour_coords) >= 3:
        polygon = Polygon(contour_coords)

        # Add polygon to the list
        polygons.append(polygon)

# Merge intersecting polygons using unary union
merged_polygon = unary_union(polygons)

# Create a GeoDataFrame with the merged polygons
if merged_polygon.geom_type == 'Polygon':
    merged_polygons_gdf = gpd.GeoDataFrame(geometry=[merged_polygon])
else:
    merged_polygons_gdf = gpd.GeoDataFrame(geometry=[merged_polygon])

# Create a new column to store the polygons
merged_polygons_gdf['geometry'] = merged_polygons_gdf.geometry

# Assign the correct CRS
merged_polygons_gdf.crs = crs

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/Poly_ACM/polygons_active_contours.shp'
merged_polygons_gdf.to_file(output_shapefile)

Code for Watershed - Failed to give good results 

In [8]:
import numpy as np
import rasterio
from skimage.measure import label
from skimage.segmentation import watershed
from skimage.segmentation import find_boundaries
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import geopandas as gpd

def polygonize_watershed(binary_image):
    # Perform watershed segmentation
    markers = label(binary_image)
    segmented = watershed(binary_image, markers)

    # Extract boundaries of the segmented regions
    boundaries = find_boundaries(segmented, connectivity=2, mode='outer')

    # Convert boundaries to polygons
    polygons = []
    for label_value in np.unique(segmented):
        if label_value == 0:  # Skip the background label
            continue
        boundary = boundaries == label_value
        contour = np.flip(np.transpose(np.where(boundary)), axis=1)  # Convert to (x, y) format
        
        # Check if the contour has at least 3 coordinate tuples
        if len(contour) < 3:
            continue
        
        polygon = Polygon(contour)
        polygons.append(polygon)

    # Merge overlapping polygons
    merged_polygon = cascaded_union(polygons)

    return merged_polygon

# Example usage
binary_image_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image_split1.tif'
with rasterio.open(binary_image_path) as src:
    binary_image = src.read(1).astype(bool)

merged_polygon = polygonize_watershed(binary_image)

# Create a GeoDataFrame with the merged polygon
merged_polygons_gdf = gpd.GeoDataFrame(geometry=[merged_polygon])

# Set the CRS of the GeoDataFrame to match the source raster
merged_polygons_gdf.crs = src.crs

# Define the output shapefile path
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/polygons_shapefile.shp'

# Save the GeoDataFrame to a shapefile
merged_polygons_gdf.to_file(output_shapefile)


/tmp/ipykernel_85773/3750396534.py:34: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  merged_polygon = cascaded_union(polygons)


Back to CCA with Marching squares - Fine tunings to imporvise 

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import label, regionprops, find_contours
from shapely.geometry import Polygon
from rasterio.crs import CRS

# Open the binary TIFF file
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image_split1.tif') as src:
    binary_image = src.read(1)
    transform = src.transform
    crs = src.crs

# Perform Connected Component Analysis
labeled_image, num_labels = label(binary_image, connectivity=2, background=0, return_num=True)
regions = regionprops(labeled_image)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['label', 'geometry'], crs=CRS.from_dict(crs))

# Iterate over the regions and extract properties
for region in regions:
    label_value = region.label
    bbox = region.bbox  # Bounding box of the region (min_row, min_col, max_row, max_col)
    centroid = region.centroid  # Centroid coordinates (row, col)
    area = region.area  # Area of the region

    # Polygonize the region using Marching Squares algorithm
    mask = (labeled_image == label_value).astype(np.uint8) * 255
    contours = find_contours(mask, level=128, fully_connected='low')

    # Iterate over the contours and create polygons
    for contour in contours:
        # Extract x and y coordinates separately
        contour_x = contour[:, 1]
        contour_y = contour[:, 0]

        # Zip x and y coordinates together
        contour_coords = zip(contour_x, contour_y)

        # Convert contour coordinates to Polygon format
        polygon = Polygon(list(contour_coords) + [(contour_x[0], contour_y[0])])

        # Add polygon to the GeoDataFrame
        polygons_gdf = polygons_gdf.append({'label': label_value, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/Poly_ACM/poly_cca_impr/polygons_srs_254.shp'
polygons_gdf.to_file(output_shapefile)


Fixed CRS issue with find contours 

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import label, regionprops, find_contours
from shapely.geometry import Polygon

# Open the binary TIFF file
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image.tif') as src:
    binary_image = src.read(1)
    transform = src.transform

# Perform Connected Component Analysis
labeled_image, num_labels = label(binary_image, connectivity=2, background=0, return_num=True)
regions = regionprops(labeled_image)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['label', 'geometry'])

# Iterate over the regions and extract properties
for region in regions:
    label_value = region.label
    bbox = region.bbox  # Bounding box of the region (min_row, min_col, max_row, max_col)
    centroid = region.centroid  # Centroid coordinates (row, col)
    area = region.area  # Area of the region

    # Polygonize the region using Marching Squares algorithm
    mask = (labeled_image == label_value).astype(np.uint8) * 255
    contours = find_contours(mask, level=128, fully_connected='low')

# Iterate over the contours and create polygons
    for contour in contours:
        # Extract x and y coordinates separately
        contour_x = contour[:, 1]
        contour_y = contour[:, 0]

        # Zip x and y coordinates together
        contour_coords = zip(contour_x, contour_y)

        # Convert contour coordinates to Polygon format
        polygon = Polygon(list(contour_coords) + [(contour_x[0], contour_y[0])])

        # Add polygon to the GeoDataFrame
        polygons_gdf = polygons_gdf.append({'label': label_value, 'geometry': polygon}, ignore_index=True)


# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/Poly_ACM/poly_cca_impr/polygons_srs_254.shp'
polygons_gdf.crs = src.crs
polygons_gdf.to_file(output_shapefile)

In [ ]:
Final script CCA & Find_Contours 

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import label, regionprops, find_contours
from shapely.geometry import Polygon

# Open the binary TIFF file
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image.tif') as src:
    binary_image = src.read(1)
    transform = src.transform

# Perform Connected Component Analysis
labeled_image, num_labels = label(binary_image, connectivity=2, background=0, return_num=True)
regions = regionprops(labeled_image)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['label', 'geometry'])

# Iterate over the regions and extract properties
for region in regions:
    label_value = region.label
    bbox = region.bbox  # Bounding box of the region (min_row, min_col, max_row, max_col)
    centroid = region.centroid  # Centroid coordinates (row, col)
    area = region.area  # Area of the region

    # Polygonize the region using Marching Squares algorithm
    mask = (labeled_image == label_value).astype(np.uint8) * 255
    contours = find_contours(mask, level=128, fully_connected='low')

    # Iterate over the contours and create polygons
    for contour in contours:
        # Extract x and y coordinates separately
        contour_x = contour[:, 1]
        contour_y = contour[:, 0]

        # Zip x and y coordinates together
        contour_coords = zip(contour_x, contour_y)

        # Convert contour coordinates to Polygon format
        polygon = Polygon(list(contour_coords) + [(contour_x[0], contour_y[0])])

        # Add polygon to the GeoDataFrame
        polygons_gdf = polygons_gdf.append({'label': label_value, 'geometry': polygon}, ignore_index=True)


# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/Poly_ACM/poly_cca_impr/polygons_srs_254.shp'
polygons_gdf.crs = src.crs  # Set the CRS of the GeoDataFrame to match the source raster
polygons_gdf.to_file(output_shapefile)


tried adaptove thresholding - Failure to get good results 

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import label, regionprops, find_contours
from shapely.geometry import Polygon

# Open the binary TIFF file
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image_adaptive_threshold.tif') as src:
    binary_image = src.read(1)
    transform = src.transform

# Perform Connected Component Analysis
labeled_image, num_labels = label(binary_image, connectivity=2, background=0, return_num=True)
regions = regionprops(labeled_image)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['label', 'geometry'])

# Iterate over the regions and extract properties
for region in regions:
    label_value = region.label
    bbox = region.bbox  # Bounding box of the region (min_row, min_col, max_row, max_col)
    centroid = region.centroid  # Centroid coordinates (row, col)
    area = region.area  # Area of the region

    # Polygonize the region using Marching Squares algorithm
    mask = (labeled_image == label_value).astype(np.uint8) * 255
    contours = find_contours(mask, level=0.5, fully_connected='low')

    # Iterate over the contours and create polygons
    for contour in contours:
        # Extract x and y coordinates separately
        contour_x = contour[:, 1]
        contour_y = contour[:, 0]

        # Zip x and y coordinates together
        contour_coords = zip(contour_x, contour_y)

        # Convert contour coordinates to Polygon format
        polygon = Polygon(list(contour_coords) + [(contour_x[0], contour_y[0])])

        # Add polygon to the GeoDataFrame
        polygons_gdf = polygons_gdf.append({'label': label_value, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/Poly_ACM/polygons_srs_254.shp'
polygons_gdf.crs = src.crs
polygons_gdf.to_file(output_shapefile)

tried Erosion - Failed to get good results 

In [5]:
import numpy as np
import rasterio
from scipy.ndimage import binary_erosion

# Load the binary image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image.tif') as src:
    binary_image = src.read(1)

# Define the structuring element (3x3 square in this case)
structuring_element = np.ones((3, 3), dtype=np.uint8)

# Perform morphological erosion
eroded_image = binary_erosion(binary_image, structure=structuring_element)

# Save the eroded image
output_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_eroded_image.tif'
with rasterio.open(output_path, 'w', **src.profile) as dst:
    dst.write(eroded_image, 1)

RasterioIOError: Dataset is closed: /home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image.tif

In [6]:
import numpy as np
import rasterio
from scipy.ndimage import binary_erosion

# Load the binary image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image.tif') as src:
    binary_image = src.read(1)
    profile = src.profile.copy()  # Make a copy of the source profile

# Define the structuring element (3x3 square in this case)
structuring_element = np.ones((3, 3), dtype=np.uint8)

# Perform morphological erosion
eroded_image = binary_erosion(binary_image, structure=structuring_element)

# Update the profile for the eroded image
profile.update(dtype=rasterio.uint8, count=1)

# Save the eroded image
output_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_eroded_image.tif'
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(eroded_image, 1)


In [24]:
import numpy as np
import rasterio
from scipy.ndimage import binary_erosion

# Load the binary image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image_adaptive_threshold.tif') as src:
    binary_image = src.read(1)
    profile = src.profile.copy()  # Make a copy of the source profile

# Define the erosion level
erosion_level = 1

# Construct the structuring element based on the erosion level
structuring_element = np.ones((2 * erosion_level + 1, 2 * erosion_level + 1), dtype=np.uint8)

# Perform morphological erosion
eroded_image = binary_erosion(binary_image, structure=structuring_element)

# Update the profile for the eroded image
profile.update(dtype=rasterio.uint8, count=1)

# Save the eroded image
output_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_eroded_image.tif'
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(eroded_image, 1)


In [11]:
from osgeo import gdal
import numpy as np
from scipy.ndimage import binary_erosion

# Define the input and output file paths
input_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif'
output_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image.tif'

# Load the input TIFF image
dataset = gdal.Open(input_path)
if dataset is None:
    print("Failed to open the input TIFF image.")
    exit()

# Read the image data
image_data = dataset.ReadAsArray()

# Select a specific band (assuming grayscale image)
band_index = 0  # Adjust this value if the grayscale band is different
image_band = image_data[band_index]

# Apply thresholding
threshold_value = 100  # Adjust this value as per your requirement
binary_image_data = np.where(image_band >= threshold_value, 255, 0)

# Perform morphological erosion
erosion_level = 1  # Adjust this value to control the level of erosion
structuring_element = np.ones((2 * erosion_level + 1, 2 * erosion_level + 1), dtype=np.uint8)
eroded_image_data = binary_erosion(binary_image_data, structure=structuring_element)

# Create the output dataset
driver = gdal.GetDriverByName('GTiff')
output_dataset = driver.Create(output_path, dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Byte)
output_dataset.GetRasterBand(1).WriteArray(eroded_image_data)

# Set the geotransform and projection
output_dataset.SetGeoTransform(dataset.GetGeoTransform())
output_dataset.SetProjection(dataset.GetProjection())

# Close the datasets
output_dataset = None
dataset = None

print("Binary image created successfully with morphological erosion.")


Binary image created successfully with morphological erosion.


In [28]:
import numpy as np
import rasterio
from skimage.filters import threshold_local

# Define the input and output file paths
input_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif'
output_path = '/home/jovyan/private/Polygonization_ACM/TifPrediction/binary_image_adaptive_threshold.tif'

# Open the input TIFF image
with rasterio.open(input_path) as src:
    # Read the image data
    image_data = src.read(1)
    image_data = image_data.astype(np.uint8)  # Convert to unsigned 8-bit integer

    # Apply adaptive thresholding
    block_size = 5  # Adjust this value as per your requirement
    binary_image_data = threshold_local(image_data, block_size, offset=20, method='gaussian')

    # Convert the binary image to 0s and 255s
    binary_image_data = (binary_image_data > image_data).astype(np.uint8) * 255

    # Copy the metadata from the input image
    profile = src.profile

    # Update the data type and count in the profile
    profile.update(dtype=rasterio.uint8, count=1)

    # Save the binary image
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(binary_image_data, 1)

print("Binary image created successfully.")

Binary image created successfully.


Started working on graph based segmentation for polygonization

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the input TIFF image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    image_data = src.read(1)

# Perform graph-based segmentation using Felzenszwalb algorithm
segments = felzenszwalb(image_data, scale=50, sigma=0.5, min_size=200)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a binary mask for the current segment
    segment_mask = segments == segment_id

    # Extract the geometry of the segment
    contours = find_contours(segment_mask, level=0.5, fully_connected='low')

    for contour in contours:
        contour_x = contour[:, 1]
        contour_y = contour[:, 0]
        polygon = Polygon(zip(contour_x, contour_y))

        # Add the polygon to the GeoDataFrame
        polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=100, sigma=0.5, min_size=200)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Convert contour coordinates to image coordinates
    contour[:, 0] = contour[:, 0] * transform.a + transform.c
    contour[:, 1] = contour[:, 1] * transform.e + transform.f

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour.tolist())

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=100, sigma=0.5, min_size=200)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation
    contour[:, 0] = transform.c + contour[:, 0] * transform.a
    contour[:, 1] = transform.f + contour[:, 1] * transform.e

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour.tolist())

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.5, min_size=200)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=100, sigma=0.5, min_size=200)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the minimum segment size threshold
min_segment_size = 100

# Set the intensity threshold for background segments
background_threshold = 100

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments below the minimum size threshold
    if segment_size < min_segment_size:
        continue

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=100, sigma=0.5, min_size=200)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the minimum segment size threshold
min_segment_size = 100

# Set the intensity threshold for background segments
background_threshold = 100

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments below the minimum size threshold
    if segment_size < min_segment_size:
        continue

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the intensity threshold for background segments
background_threshold = 20

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1, min_size=200)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the minimum segment size threshold
min_segment_size = 100

# Set the maximum segment size threshold
max_segment_size = 1000

# Set the intensity threshold for background segments
background_threshold = 100

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments below the minimum size threshold or above the maximum size threshold
    if segment_size < min_segment_size or segment_size > max_segment_size:
        continue

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


Final version below - Tuned Graph Based segmentation for polygonization. 

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the maximum segment size threshold
max_segment_size = 1000

# Set the intensity threshold for background segments
background_threshold = 100

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments above the maximum size threshold
    if segment_size > max_segment_size:
        continue

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import find_contours
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1, min_size=100, max_size=1000)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the intensity threshold for background segments
background_threshold = 100

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments based on min and max size thresholds
    if segment_size < 100 or segment_size > 1000:
        continue

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import regionprops
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the minimum and maximum segment size thresholds
min_segment_size = 100
max_segment_size = 1000

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments below the minimum size threshold or above the maximum size threshold
    if segment_size < min_segment_size or segment_size > max_segment_size:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)


Working Final Code for Post processing - max segment, thresholding , solodity implemented below - Min segment removed since dint improve segmentation much.

In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import find_contours
from skimage.measure import regionprops
from shapely.geometry import Polygon

# Open the grayscale image
with rasterio.open('/home/jovyan/private/Polygonization_ACM/TifPrediction/1.tif') as src:
    grayscale_image = src.read(1)
    transform = src.transform

# Perform graph-based segmentation
segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1)

# Create an empty GeoDataFrame for the polygons
polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

# Set the maximum segment size threshold
max_segment_size = 1000

# Set the intensity threshold for background segments
background_threshold = 100

# Set the solidity threshold for irregular fragments
solidity_threshold = 0.7

# Iterate over the segments and create polygons
for segment_id in np.unique(segments):
    # Create a mask for the current segment
    segment_mask = (segments == segment_id)

    # Calculate the size of the segment (number of pixels)
    segment_size = np.sum(segment_mask)

    # Skip segments above the maximum size threshold
    if segment_size > max_segment_size:
        continue

    # Extract the intensity values of the segment
    segment_intensities = grayscale_image[segment_mask]

    # Check if the segment is predominantly background based on intensity threshold
    if np.mean(segment_intensities) < background_threshold:
        continue

    # Find the contour of the segment mask
    contour = find_contours(segment_mask, level=0.5)[0]

    # Adjust the contour coordinates to match the image orientation and transformation
    contour_coords = []
    for coord in contour:
        x, y = rasterio.transform.xy(transform, coord[0], coord[1])
        contour_coords.append([x, y])

    # Create a Polygon from the contour coordinates
    polygon = Polygon(contour_coords)

    # Calculate the solidity of the segment
    props = regionprops(segment_mask.astype(int))[0]
    solidity = props.solidity

    # Check if the segment has irregular fragments based on solidity threshold
    if solidity < solidity_threshold:
        continue

    # Add polygon to the GeoDataFrame
    polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

# Export polygons as a shapefile
output_shapefile = '/home/jovyan/private/Polygonization_ACM/TifPrediction/output_polygons_gt.shp'
polygons_gdf.to_file(output_shapefile)
